In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# 回路を構築する

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>パッケージバージョン</b></summary>

このページのコードは、以下の要件を使用して開発されました。
これらのバージョンまたはそれ以降のバージョンの使用を推奨します。

```
qiskit[all]~=2.3.0
```
</details>
このページでは、Qiskit SDKの[`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit)クラスについて詳しく見ていきます。量子回路を作成するために使用できるより高度なメソッドも含まれています。
## 量子回路とは何か？
シンプルな量子回路は、量子ビットのコレクションと、それらの量子ビットに作用する命令のリストです。デモンストレーションのために、次のセルは2つの新しい量子ビットを持つ新しい回路を作成し、回路の[`qubits`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qubits)属性を表示します。これは、最下位ビット$q_0$から最上位ビット$q_n$まで順番に並んだ[`Qubits`](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.Qubit)のリストです。

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.qubits

[<Qubit register=(2, "q"), index=0>, <Qubit register=(2, "q"), index=1>]

Multiple `QuantumRegister` and `ClassicalRegister` objects can be combined to create a circuit. Every [`QuantumRegister`](/docs/api/qiskit/circuit#qiskit.circuit.QuantumRegister) and [`ClassicalRegister`](/docs/api/qiskit/circuit#qiskit.circuit.ClassicalRegister) can also be named.

In [2]:
from qiskit.circuit import QuantumRegister, ClassicalRegister

qr1 = QuantumRegister(2, "qreg1")  # Create a QuantumRegister with 2 qubits
qr2 = QuantumRegister(1, "qreg2")  # Create a QuantumRegister with 1 qubit
cr1 = ClassicalRegister(3, "creg1")  # Create a ClassicalRegister with 3 cbits

combined_circ = QuantumCircuit(
    qr1, qr2, cr1
)  # Create a quantum circuit with 2 QuantumRegisters and 1 ClassicalRegister
combined_circ.qubits

[<Qubit register=(2, "qreg1"), index=0>,
 <Qubit register=(2, "qreg1"), index=1>,
 <Qubit register=(1, "qreg2"), index=0>]

複数の`QuantumRegister`オブジェクトと`ClassicalRegister`オブジェクトを組み合わせて回路を作成できます。すべての[`QuantumRegister`](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.QuantumRegister)と[`ClassicalRegister`](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.ClassicalRegister)には名前を付けることもできます。

In [3]:
desired_qubit = qr2[0]  # Qubit 0 of register 'qreg2'

print("Index:", combined_circ.find_bit(desired_qubit).index)
print("Register:", combined_circ.find_bit(desired_qubit).registers)

Index: 2
Register: [(QuantumRegister(1, 'qreg2'), 0)]


Adding an instruction to the circuit appends the instruction to the circuit's [`data`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#data) attribute. The following cell output shows `data` is a list of [`CircuitInstruction`](/docs/api/qiskit/qiskit.circuit.CircuitInstruction) objects, each of which has an `operation` attribute, and a `qubits` attribute.

In [4]:
qc.x(0)  # Add X-gate to qubit 0
qc.data

[CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(<Qubit register=(2, "q"), index=0>,), clbits=())]

回路の[`find_bit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.find_bit)メソッドとその属性を使用して、量子ビットのインデックスとレジスターを見つけることができます。

In [5]:
qc.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/43a57258-3e33-4071-8a48-2bf127c8a5be-0.svg" alt="Output of the previous code cell" />

Circuit instruction objects can contain "definition" circuits that describe the instruction in terms of more fundamental instructions. For example, the [X-gate](/docs/api/qiskit/qiskit.circuit.library.XGate) is defined as a specific case of the [U3-gate](/docs/api/qiskit/qiskit.circuit.library.U3Gate), a more general single-qubit gate.

In [6]:
# Draw definition circuit of 0th instruction in `qc`
qc.data[0].operation.definition.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/653e2427-e301-4d2f-84de-1959185ace8e-0.svg" alt="Output of the previous code cell" />

回路に命令を追加すると、回路の[`data`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#data)属性に命令が追加されます。次のセル出力は、`data`が[`CircuitInstruction`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.CircuitInstruction)オブジェクトのリストであり、それぞれが`operation`属性と`qubits`属性を持つことを示しています。

In [7]:
from qiskit.circuit.library import HGate

qc = QuantumCircuit(1)
qc.append(
    HGate(),  # New HGate instruction
    [0],  # Apply to qubit 0
)
qc.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/66813cae-9841-47ea-96b7-8fd7b82e9759-0.svg" alt="Output of the previous code cell" />

To combine two circuits, use the [`compose`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#compose) method. This accepts another [`QuantumCircuit`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit) and an optional list of qubit mappings.

<Admonition type="note">
    The [`compose`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#compose) method returns a new circuit and does **not** mutate either circuit it acts on. To mutate the circuit on which you're calling the [`compose`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#compose) method, use the argument `inplace=True`.
</Admonition>

In [8]:
qc_a = QuantumCircuit(4)
qc_a.x(0)

qc_b = QuantumCircuit(2, name="qc_b")
qc_b.y(0)
qc_b.z(1)

# compose qubits (0, 1) of qc_a to qubits (1, 3) of qc_b respectively
combined = qc_a.compose(qc_b, qubits=[1, 3])
combined.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/29152dfa-2275-4bc4-aadb-82185b9e0e86-0.svg" alt="Output of the previous code cell" />

この情報を表示する最も簡単な方法は、回路の可視化を返す[`draw`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#draw)メソッドを使用することです。量子回路を表示するさまざまな方法については、[回路の可視化](./visualize-circuits)を参照してください。

In [9]:
inst = qc_b.to_instruction()
qc_a.append(inst, [1, 3])
qc_a.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/81b682dd-45cb-4492-809e-d9e8ebbf5600-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/43a57258-3e33-4071-8a48-2bf127c8a5be-0.svg)

回路命令オブジェクトには、より基本的な命令の観点から命令を記述する「定義」回路を含めることができます。たとえば、[Xゲート](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.XGate)は、より一般的な単一量子ビットゲートである[U3ゲート](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.U3Gate)の特定のケースとして定義されています。

In [10]:
gate = qc_b.to_gate().control()
qc_a.append(gate, [0, 1, 3])
qc_a.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/ed362e64-d6a4-4dfd-a5cf-5e6bdc7a81b5-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/653e2427-e301-4d2f-84de-1959185ace8e-0.svg)

命令と回路は、どちらもビットと量子ビットの操作を記述するという点で似ていますが、異なる目的があります:

- 命令は固定として扱われ、そのメソッドは通常、新しい命令を返します(元のオブジェクトを変更しません)。
- 回路は、多くのコード行にわたって構築されるように設計されており、[`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit)メソッドは多くの場合、既存のオブジェクトを変更します。
### 回路の深さとは何か？
量子回路の[depth()](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.depth)は、並列実行される量子ゲートの「レイヤー」の数の尺度であり、回路によって定義される計算を完了するために必要です。量子ゲートの実装には時間がかかるため、回路の深さは、量子コンピューターが回路を実行するのにかかる時間にほぼ対応します。したがって、回路の深さは、量子回路がデバイスで実行できるかどうかを測定するために使用される重要な量の1つです。

このページの残りの部分では、量子回路を操作する方法を説明します。
## 回路を構築する
[`QuantumCircuit.h`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#h)や[`QuantumCircuit.cx`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#cx)などのメソッドは、特定の命令を回路に追加します。より一般的に回路に命令を追加するには、[`append`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#append)メソッドを使用します。これは、命令と、命令を適用する量子ビットのリストを取ります。サポートされている命令のリストについては、[回路ライブラリAPIドキュメント](https://docs.quantum.ibm.com/api/qiskit/circuit_library)を参照してください。

In [11]:
qc_a.decompose().draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/3c0633db-929b-4428-a888-7a3d493bd6dd-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/66813cae-9841-47ea-96b7-8fd7b82e9759-0.svg)

2つの回路を結合するには、[`compose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#compose)メソッドを使用します。これは、別の[`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit)とオプションの量子ビットマッピングのリストを受け入れます。

> **Note:** [`compose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#compose)メソッドは新しい回路を返し、作用する回路を変更**しません**。[`compose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#compose)メソッドを呼び出している回路を変更するには、引数`inplace=True`を使用します。

In [12]:
qc1 = QuantumCircuit(2, 2)
qc1.measure(0, 1)
qc1.draw("mpl", cregbundle=False)

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/0cdb2273-0.svg" alt="Output of the previous code cell" />

In [13]:
qc2 = QuantumCircuit(2)
qc2.measure_all()
qc2.draw("mpl", cregbundle=False)

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/6f33698c-0.svg" alt="Output of the previous code cell" />

In [14]:
qc3 = QuantumCircuit(2)
qc3.x(1)
qc3.measure_active()
qc3.draw("mpl", cregbundle=False)

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/ca3f225f-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/ed362e64-d6a4-4dfd-a5cf-5e6bdc7a81b5-0.svg)

何が起こっているかを確認するには、[`decompose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#decompose)メソッドを使用して、各命令をその定義に展開できます。

> **Note:** [`decompose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#decompose)メソッドは新しい回路を返し、作用する回路を変更**しません**。

In [15]:
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.circuit import Parameter

angle = Parameter("angle")  # undefined number

# Create and optimize circuit once
qc = QuantumCircuit(1)
qc.rx(angle, 0)
qc = generate_preset_pass_manager(
    optimization_level=3, basis_gates=["u", "cx"]
).run(qc)

qc.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/a580552c-d585-4047-99f0-32aafd06e4f3-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/3c0633db-929b-4428-a888-7a3d493bd6dd-0.svg)

<span id="measure-qubits"></span>
## 量子ビットを測定する
測定は、個々の量子ビットの状態をサンプリングし、結果を古典レジスターに転送するために使用されます。[Sampler](./primitives#sampler)プリミティブに回路を送信する場合、測定が必要であることに注意してください。ただし、[Estimator](./primitives#estimator)プリミティブに送信される回路には測定を含めてはいけません。

量子ビットは、3つのメソッドを使用して測定できます: [`measure`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.measure)、[`measure_all`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#measure_all)、[`measure_active`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#measure_active)。測定結果を可視化する方法については、[結果の可視化](./visualize-results)ページを参照してください。

1. `QuantumCircuit.measure` : 最初の引数の各量子ビットを、2番目の引数として指定された古典ビットに測定します。このメソッドは、測定結果の保存場所を完全に制御できます。

2. `QuantumCircuit.measure_all` : 引数を取らず、事前定義された古典ビットのない量子回路に使用できます。古典ワイヤーを作成し、測定結果を順番に保存します。たとえば、量子ビット$q_i$の測定は古典ビット$meas_i$に保存されます。また、測定の前にバリアを追加します。

3. `QuantumCircuit.measure_active` : `measure_all`と似ていますが、操作がある量子ビットのみを測定します。

In [16]:
circuits = []
for value in range(100):
    circuits.append(qc.assign_parameters({angle: value}))

circuits[0].draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/85af6231-921a-4130-99d3-f6998f761df8-0.svg" alt="Output of the previous code cell" />

You can find a list of a circuit's undefined parameters in its `parameters` attribute.

In [17]:
qc.parameters

ParameterView([Parameter(angle)])

### Change a parameter's name

By default, parameter names for a parameterized circuit are prefixed by `x`- for example, `x[0]`. You can change the names after they are defined, as shown in the following example.

In [18]:
from qiskit.circuit.library import z_feature_map
from qiskit.circuit import ParameterVector

# Define a parameterized circuit with default names
# For example, x[0]
circuit = z_feature_map(2)

# Set new parameter names
# They will now be prefixed by `hi` instead
# For example, hi[0]
training_params = ParameterVector("hi", 2)

# Assign parameter names to the quantum circuit
circuit = circuit.assign_parameters(parameters=training_params)

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/ca3f225f-0.svg)

## パラメータ化された回路

多くの近期量子アルゴリズムは、量子回路の多くのバリエーションを実行することを含みます。大規模な回路の構築と最適化は計算コストが高くなる可能性があるため、Qiskitは**パラメータ化**された回路をサポートしています。これらの回路には未定義のパラメータがあり、その値は回路を実行する直前まで定義する必要がありません。これにより、回路の構築と最適化をメインプログラムループの外に移動できます。次のセルは、パラメータ化された回路を作成して表示します。